In [1]:
%load_ext lab_black

In [2]:
from collections import namedtuple
from copy import deepcopy
from dataclasses import dataclass
from puzzles import load

In [3]:
s = load(22)
print(s.strip())

Hit Points: 71
Damage: 10


In [4]:
@dataclass
class Player:
    hp: int = 50
    mana: int = 500
    armor: int = 0
    shield_left: int = 0
    recharge_left: int = 0

    recharge_time: int = 5
    recharge_add: int = 101

    shield_time: int = 6
    shield_add: int = 7

    def recharge_tick(self):
        if self.recharge_left > 0:
            self.recharge_left -= 1
            self.mana += self.recharge_add

    def shield_tick(self):
        if self.shield_left > 0:
            self.shield_left -= 1
        elif self.shield_left == 0:
            self.armor = 0

    def __repr__(self):
        return (
            f"Player(hp={self.hp}, mana={self.mana}, armor={self.armor}, "
            f"shield_left={self.shield_left}, recharge_left={self.recharge_left})"
        )


@dataclass
class Boss:
    hp: int = 71
    damage: int = 10
    poison_left: int = 0

    poison_time: int = 6
    poison_add: int = 3

    def poison_tick(self):
        if self.poison_left > 0:
            self.hp -= self.poison_add
            self.poison_left -= 1

    def __repr__(self):
        return (
            f"Boss(hp={self.hp}, damage={self.damage}, poison_left={self.poison_left})"
        )


def magic_missile(p, b):
    player, boss = deepcopy(p), deepcopy(b)
    player.mana -= 53
    boss.hp -= 4
    return player, boss


def drain(p, b):
    player, boss = deepcopy(p), deepcopy(b)
    player.hp += 2
    player.mana -= 73
    boss.hp -= 2
    return player, boss


def shield(p, b):
    player, boss = deepcopy(p), deepcopy(b)
    player.shield_left = player.shield_time
    player.armor = player.shield_add
    player.mana -= 113
    return player, boss


def poison(p, b):
    player, boss = deepcopy(p), deepcopy(b)
    player.mana -= 173
    boss.poison_left = boss.poison_time
    return player, boss


def recharge(p, b):
    player, boss = deepcopy(p), deepcopy(b)
    player.mana -= 229
    player.recharge_left = player.recharge_time
    return player, boss


def tick(p, b):
    if p.shield_left > 0:
        p.shield_left -= 1

    return p, b

```python
-- Player turn --
- Player has 10 hit points, 0 armor, 250 mana
- Boss has 14 hit points
Player casts Recharge.

-- Boss turn --
- Player has 10 hit points, 0 armor, 21 mana
- Boss has 14 hit points
Recharge provides 101 mana; its timer is now 4.
Boss attacks for 8 damage!
```

In [33]:
from enum import Enum


class GameState(Enum):
    in_progress: int = 0
    won: int = 1
    lost: int = 2


class GameNode:
    def __init__(self, player, boss, spells, game_state=GameState.in_progress, total_mana_spent=0):
        self.player = player
        self.boss = boss
        self.spells = spells
        self.game_state = game_state
        self.total_mana_spent = total_mana_spent
        self.nodes = None

    def take_turn(self):
        if self.game_state != GameState.in_progress:
            return

        # Player turn

        self.boss.poison_tick()
        if self.boss.hp <= 0:
            self.game_state = GameState.won
            return

        self.player.shield_tick()
        self.player.recharge_tick()

        self.nodes = []
        for cast_another_spell in self.spells:

            player, boss = cast_another_spell(self.player, self.boss)

            # the spell was invalid, out of mana
            if player.mana < 0:
                continue

            if boss.hp <= 0:
                game_state = GameState.won
            else:
                # Boss turn

                game_state = GameState.in_progress
                boss.poison_tick()

                if boss.hp <= 0:
                    game_state = GameState.won
                else:
                    player.shield_tick()
                    player.recharge_tick()

                    player.hp -= boss.damage - player.armor
                    if player.hp <= 0:
                        game_state = GameState.lost

            gn = GameNode(player, boss, self.spells, game_state=game_state)
            self.nodes.append(gn)

    def __repr__(self):
        return f"{self.game_state}\n\t{self.player}\n\t{self.boss}"

In [37]:
def dsf(node, level=0):
    node.take_turn()

    if node.nodes is None or len(node.nodes) == 0:
        return

    for child in node.nodes:
        if child.game_state != GameState.in_progress:
            continue
        try:
            dsf(child, level + 1)
        except:
            print(level)
            print(node.nodes)
            raise

In [39]:
game_tree = GameNode(
    Player(), Boss(), spells=[magic_missile, drain, shield, poison, recharge]
)

In [1]:
import pandas as pd

In [15]:
df = pd.DataFrame(data={
    'floor': [1,3,5,7,9,1],
    'floors_total': [5,5,5,7,10,1],
})

df['tp'] = 'Другой'
df.loc[df['floor'] == df['floors_total'], 'tp'] = 'Последний'
df.loc[df['floor'] == 1, 'tp'] = 'Первый'

# df['tp'] = df['tp'].where(df['floor'] > 1, 'Первый')
# df['tp'] = df['tp'].where(df['floor'] < df['floors_total'], 'Последний')

df


,floor,floors_total,tp
0,1,5,Первый
1,3,5,Другой
2,5,5,Последний
3,7,7,Последний
4,9,10,Другой
5,1,1,Первый


In [ ]:
dsf(game_tree)

In [ ]:
print('done!')